In [104]:
####데이터 이상$$$$$$ 사용x
#대조약 조회
#약의 성분 추출
import requests
from urllib.parse import unquote

SERVICE_KEY = "YKH%2FsKWkIH6%2FFfZlcEhCFOL8m%2B2j2E802xPlLD2I1736c2gNNAuxZ99KawQt0XpAyBZ22o9LWQA1Piu8D1LTng%3D%3D" 
serviceKey = unquote(SERVICE_KEY)  # 디코딩
base_url = "https://apis.data.go.kr/1471000/MdcCompDrugInfoService04"
endpoint = "getMdcCompDrugList04"

def get_medicine_info_ingre(pill):
    params = {
        "serviceKey": serviceKey,
        "pageNo": 1,
        "numOfRows": 10,
        "type": "json",
        "item_name": pill,
    }
    url = f"{base_url}/{endpoint}"
    res = requests.get(url, params=params)
    result=res.json()

    return result['body']

In [105]:
####데이터 이상$$$$$$ 사용x
pill_ingre=set()  #모든 약의 성분 총 집합
dupli=[]          #중복 약물

for pill in medicine:
    result=get_medicine_info_ingre(pill)  #약의 성분
    
    if 'items' in result:
        ingredient=result['items'][0]['item']['INGR_NAME']
        
        #성분이 여러개인 경우
        if "/" in ingredient:
            n_ingre=ingredient.split("/")
            for n in n_ingre:
                if n in pill_ingre:
                    dupli.append(n)
                pill_ingre.add(n)
        
        #단일 성분
        else:
            pill_ingre.add(ingredient)

In [ ]:
################## 여기부터 #####################

In [120]:
#사용자 복용 약 DB에서 가져오기
medicine=['메토트렉세이트', '아스피린', '아스피린']

In [121]:
#병용금기 정보
import requests
from urllib.parse import unquote

SERVICE_KEY = "YKH%2FsKWkIH6%2FFfZlcEhCFOL8m%2B2j2E802xPlLD2I1736c2gNNAuxZ99KawQt0XpAyBZ22o9LWQA1Piu8D1LTng%3D%3D" 
serviceKey = unquote(SERVICE_KEY)  
base_url = "https://apis.data.go.kr/1471000/DURIrdntInfoService03"
endpoint = "getUsjntTabooInfoList02"

#INGR_CODE(주성분 코드), MIXTURE_INGR_CODE(상대 성분 코드)
#ORI(원래 성분의 제품 목록), MIXTURE_ORI(상대 성분 제품 목록)
#PROHBT_CONTENT(금기 이유)
def contrain_ingre(ingredient):
    params = {
        "serviceKey": serviceKey,
        "pageNo": 1,
        "numOfRows": 10,
        "type": "json",
        "ingrKorName": ingredient,
    }

    url = f"{base_url}/{endpoint}"
    res = requests.get(url, params=params)
    result=res.json()
    return result
#result['body']

In [145]:
#병용금기 성분
contrain_list=[] #약 성분, 충돌 성분, 충돌 원인
collision_set=set() #약 충돌 성분
duplicate_set=set() #약 중복 성분

for i in medicine:
    result=contrain_ingre(i)  #약의 성분과의 충돌 성분, 충돌 원인
    if 'items' in result['body']:
        #충돌 성분
        contrain=result['body']['items'][0]['item']['MIXTURE_INGR_KOR_NAME']
        #충돌 원인
        prohbt=result['body']['items'][0]['item']['PROHBT_CONTENT'].strip()
        for ingre1, ingre2, w in contrain_list:
            #충돌 성분
            if ingre2 == contrain or ingre1 == contrain:
                collision_set.add(contrain)
            #중복 성분
            if (i, contrain) in [(a, b) for a, b, _ in contrain_list]:
                duplicate_set.add(i)
        contrain_list.append((i,contrain,prohbt))

#print(contrain_set)

In [154]:
#충돌 약(주의사항 출력할 내용)
warning_set=set()

for ingre1, ingre2, warn in contrain_list:
    if ingre1 in duplicate_set:
        warning_set.add((ingre1,"[중복 성분]", warn))
    if ingre1 in collision_set:
        warning_set.add((ingre1,"[충돌 성분]", warn))
        
warning_set

{('메토트렉세이트',
  '[충돌 성분]',
  '엽산 의존 대사 경로에서 구내염, 골수억제 및 신경독성과 같은 독성을 잠재적으로 유발하는 메토트렉세이트의 효과 증가'),
 ('아스피린', '[중복 성분]', '혈액학적 독성')}

In [158]:
#위험도 지수
cnt=len(pill_ingre) #총 성분 수

risk_rate=int((len(collision_set)+len(duplicate_set))/(cnt*(cnt-1)/2) * 100)

In [159]:
risk_rate

66